# Prep

Setting up some prior functionality

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.5.1+cu101 True


# Load a model

First we have to decide if our model should be pretrained. 

This greatly depends on the size of a dataset. Smaller datasets rely more on finetuning. 

In [2]:
pretrained = True

if pretrained:
    # Get pretrained weights
    checkpoint = torch.hub.load_state_dict_from_url(
                url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
                map_location='cpu',
                check_hash=True)

    # Remove class weights
    del checkpoint["model"]["class_embed.weight"]
    del checkpoint["model"]["class_embed.bias"]

    # SaveOGH
    torch.save(checkpoint,
               'detr-r50_no-class-head.pth')

# Dataset

Our dataset should be loadable as a COCO format

This allows us to use the pycocotools to load the data dict for the main python script

In [3]:
# alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py
dataset_file = "coco_petct" 

# should lead to a directory with a train2017 and val2017 folder as well as an annotations folder
#dataDir='/COCO_dataset/' 
#dataDir = '/media/storage/Joy/datasets/DETR_MIP/FDG-PET-CT-Lesions/'
dataDir = '/gpfs/fs0/data/stanford_data/petct/DETR_MIP/FDG-PET-CT-Lesions/'

# this int should be the actual number of classes + 1 (for no class)
num_classes = 1 

outDir = 'outputs'
resume = "detr-r50_no-class-head.pth" if pretrained else ""

In [7]:
!ls $dataDir

annotations  images


In [16]:
data_list = [0,1,2,3,4,5,6,7,8]
new_list = [data_list[i:i+3] for i in range(0, len(data_list), 3)]
new_list

list(range(0,5))

import random
 
# initializing lists
# test_list1 = [6, 4, 8, 9, 10]
# test_list2 = [1, 2, 3, 4, 5]
test_list1 = [{'id':1},{'id':2},{'id':3}]
test_list2 = [{'id':1},{'id':2},{'id':3}]
 
# printing lists
print(f"The original list 1 : {test_list1}")
print(f"The original list 2 : {test_list2}")
 
# Shuffle two lists with same order
# Using zip() + * operator + shuffle()
temp = list(zip(test_list1, test_list2))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
res1, res2 = list(res1), list(res2)
 
# Printing result
print(f"List 1 after shuffle :  {res1}")
print(f"List 2 after shuffle :  {res2}")

The original list 1 : [{'id': 1}, {'id': 2}, {'id': 3}]
The original list 2 : [{'id': 1}, {'id': 2}, {'id': 3}]
List 1 after shuffle :  [{'id': 2}, {'id': 3}, {'id': 1}]
List 2 after shuffle :  [{'id': 2}, {'id': 3}, {'id': 1}]


# Training

We use the main.py script to run our training

In [ ]:
!python main.py \
  --dataset_file $dataset_file \
  --coco_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs 1

# Results

Quick and easy overview of the training results

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path(outDir)]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)   